In [165]:
import pandas as pd
import numpy as np
from faker import Faker
import hashlib

In [166]:
# %pip install faker

In [167]:
df = pd.read_excel("data/raw/arquivo_anonimizado.xlsx")

In [168]:
faker = Faker("pt_BR")

quantidade_de_nomes = df.shape[0]
nomes_completos = [faker.name() for _ in range(quantidade_de_nomes)]

df["nome_completo"] = nomes_completos

In [169]:
df.head(5).T

,0,1,2,3,4
Unnamed: 0,0,1,2,3,4
matriculaDRE,0,1,2,3,4
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Informática,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por abandono,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 062...,2000/1 - MAA123 Algebra para Informatica - 009...,2000/1 - MAA123 Algebra para Informatica - 040...,2000/1 - MAA123 Algebra para Informatica - 081...
crPorPeriodo,2000/1 - 7.6\n2000/2 - 7.1\n2001/1 - 5.9\n2001...,2000/1 - 7.6\n2000/2 - 6.5\n2001/1 - 5.1\n2001...,2000/1 - 3.3\n2000/2 - 4.7\n2001/1 - 0.7,2000/1 - 6.3\n2000/2 - 5.1\n2001/1 - 2.8\n2001...,2000/1 - 8.3\n2000/2 - 7\n2001/1 - 7.2\n2001/2...


In [170]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2817 entries, 0 to 2816
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 2817 non-null   int64  
 1   matriculaDRE               2817 non-null   int64  
 2   periodoIngressoUFRJ        2817 non-null   object 
 3   cursoIngressoUFRJ          2817 non-null   object 
 4   cursoAtual                 2817 non-null   object 
 5   periodoIngressoCursoAtual  2817 non-null   object 
 6   situacaoMatriculaAtual     2817 non-null   object 
 7   sexo                       2817 non-null   object 
 8   disciplinasCursadas        2650 non-null   object 
 9   crPorPeriodo               2644 non-null   object 
 10  craPorPeriodo              2642 non-null   object 
 11  periodosIntegralizados     2817 non-null   int64  
 12  cargaHorariaAcumulada      2817 non-null   int64  
 13  CRA                        2654 non-null   float

# Processamento


Algumas colunas não trazem ganho de informação, como por exemplo reprovações já estão na coluna disciplinas cursadas, e periodo com cr menor que 3, pode ser analisar na coluna CR por periodo


In [171]:
df.drop(
    [
        "Unnamed: 0",
        "CRA",
        "periodosCRMenor3",
        "periodosTrancados",
        "periodosCancelados",
        "reprovacoes",
    ],
    axis=1,
    inplace=True,
)

Removendo alunos que não possui disciplinas cursas, pois se não há disciplinas, não há metricas a serem analisadas.


In [172]:
# df.dropna(subset=["disciplinasCursadas"], inplace=True)
df = df[df['periodoIngressoUFRJ'] != '2022/2']

## Normaliazndo Colunas Multivaloradas


In [173]:
def gerar_periodo_temp(row):
    if pd.isna(row['disciplinasCursadas']) and pd.isna(row['crPorPeriodo']) and pd.isna(row['craPorPeriodo']):
        return row['periodoIngressoUFRJ']
    else:
        return None

df['periodo_temp'] = df.apply(gerar_periodo_temp, axis=1)


In [174]:
disciplina_nan = df['disciplinasCursadas'].isna()
cr_nan = df['crPorPeriodo'].isna()
cra_nan = df['craPorPeriodo'].isna()
df[disciplina_nan & cr_nan & cra_nan].T

,50,95,118,125,167,206,210,213,215,219,...,2401,2465,2548,2571,2575,2587,2590,2605,2691,2746
matriculaDRE,50,95,118,125,167,206,210,213,215,219,...,2401,2465,2548,2571,2575,2587,2590,2605,2691,2746
periodoIngressoUFRJ,2000/1,2000/2,2000/2,2000/2,2001/1,2001/2,2001/2,2001/2,2001/2,2001/2,...,2019/1,2019/2,2020/1,2020/1,2020/2,2020/2,2020/2,2020/2,2021/2,2022/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,...,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Ciência da Computação,Ciência da Computação
cursoAtual,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,...,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Ciência da Computação,Ciência da Computação,Ciência da Computação,Ciência da Computação,Ciência da Computação,Ciência da Computação,Ciência da Computação,Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/2,2000/2,2000/2,2001/1,2001/2,2001/2,2001/2,2001/2,2001/2,...,2019/1,2019/2,2020/1,2020/1,2021/1,2020/2,2020/2,2020/2,2021/2,2022/1
situacaoMatriculaAtual,Cancelada por transferência,Cancelada por abandono,Cancelamento por ultrapassagem do prazo máximo...,Cancelamento por ultrapassagem do prazo máximo...,Cancelada por abandono,Cancelada por abandono,Cancelada por abandono,Cancelada por abandono,Cancelada por abandono,Cancelada por abandono,...,Cancelamento a pedido,Exclusão Lógica,Cancelamento por opção de curso,Cancelamento a pedido,Cancelamento a pedido,Exclusão Lógica,Exclusão Lógica,Cancelamento a pedido,Cancelamento a pedido,Exclusão Lógica
sexo,M,M,M,F,M,M,M,M,M,M,...,M,M,M,M,F,M,M,F,M,M
disciplinasCursadas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
crPorPeriodo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
craPorPeriodo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### CR por Periodo


In [175]:
df["crPorPeriodo"] = df["crPorPeriodo"].astype(str)
df["crPorPeriodo"] = df["crPorPeriodo"].apply(
    lambda x: [tuple(item.split(" - ")) for item in x.split("\n")]
)
df_exploded = df.explode("crPorPeriodo")
df_exploded.head(5).T

,0,0,0,0,0
matriculaDRE,0,0,0,0,0
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...
crPorPeriodo,"(2000/1, 7.6)","(2000/2, 7.1)","(2001/1, 5.9)","(2001/2, 6.8)","(2002/1, 6.3)"
craPorPeriodo,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...


In [176]:
df_crPeriodo = pd.DataFrame(
    df_exploded["crPorPeriodo"].tolist(),
    index=df_exploded.index,
    columns=("periodo", "cr_periodo"),
)
df_crPeriodo[["matriculaDRE", "periodo_temp"]] = df_exploded[["matriculaDRE", "periodo_temp"]]
df_crPeriodo

,periodo,cr_periodo,matriculaDRE,periodo_temp
0,2000/1,7.6,0,None
0,2000/2,7.1,0,None
0,2001/1,5.9,0,None
0,2001/2,6.8,0,None
0,2002/1,6.3,0,None
...,...,...,...,...
2775,2022/1,7.5,2775,None
2776,2022/1,7.8,2776,None
2777,2022/1,7.9,2777,None
2778,2022/1,6.7,2778,None


In [177]:
df_crPeriodo[df_crPeriodo['matriculaDRE'] == 50]

,periodo,cr_periodo,matriculaDRE,periodo_temp
50,nan,None,50,2000/1


In [178]:
def preencher_periodo(row):
    if row['periodo'] == 'nan' :
        return row['periodo_temp']
    else:
        return row['periodo'] 

df_crPeriodo['periodo']  = df_crPeriodo.apply(preencher_periodo, axis=1)
df_crPeriodo.drop('periodo_temp', inplace=True, axis=1)

In [179]:
df_crPeriodo[df_crPeriodo['matriculaDRE'] == 50]

,periodo,cr_periodo,matriculaDRE
50,2000/1,None,50


### CRA por Periodo


In [180]:
df["craPorPeriodo"] = df["craPorPeriodo"].astype(str)
df["craPorPeriodo"] = df["craPorPeriodo"].apply(
    lambda x: [tuple(item.split(" - ")) for item in x.split("\n")]
)
df_exploded = df.explode("craPorPeriodo")
df_exploded.head(5).T

,0,0,0,0,0
matriculaDRE,0,0,0,0,0
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...
crPorPeriodo,"[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ..."
craPorPeriodo,"(2000/1, 7.6)","(2000/2, 7.4)","(2001/1, 6.9)","(2001/2, 6.9)","(2002/1, 6.8)"


In [181]:
df_craPeriodo = pd.DataFrame(
    df_exploded["craPorPeriodo"].tolist(),
    index=df_exploded.index,
    columns=("periodo", "cr_acumulado"),
)
df_craPeriodo[["matriculaDRE", "periodo_temp"]] = df_exploded[["matriculaDRE", "periodo_temp"]]
df_craPeriodo

,periodo,cr_acumulado,matriculaDRE,periodo_temp
0,2000/1,7.6,0,None
0,2000/2,7.4,0,None
0,2001/1,6.9,0,None
0,2001/2,6.9,0,None
0,2002/1,6.8,0,None
...,...,...,...,...
2775,2022/1,7.5,2775,None
2776,2022/1,7.8,2776,None
2777,2022/1,7.9,2777,None
2778,2022/1,6.7,2778,None


In [182]:
df_craPeriodo[df_craPeriodo['matriculaDRE'] == 50]

,periodo,cr_acumulado,matriculaDRE,periodo_temp
50,nan,None,50,2000/1


In [183]:
df_craPeriodo['periodo']  = df_craPeriodo.apply(preencher_periodo, axis=1)
df_craPeriodo.drop('periodo_temp', inplace=True, axis=1)

In [184]:
df_craPeriodo[df_craPeriodo['matriculaDRE'] == 50]

,periodo,cr_acumulado,matriculaDRE
50,2000/1,None,50


### Disciplinas Cursadas


Nessa seção vamos separar a lista de disciplinas em varias colunas, uma para cada disciplina


In [185]:
def processar_disciplinas(dados):
    dados = dados.astype(str)
    return dados.apply(lambda x: [split_disciplina(item) for item in x.split("\n")])


def split_disciplina(disciplina):
    detalhes_disciplinas = disciplina.split(" - ")
    if len(detalhes_disciplinas) == 5:
        detalhes_disciplinas[1] = " - ".join(detalhes_disciplinas[1:3])
        del detalhes_disciplinas[2]
    return tuple(detalhes_disciplinas)

In [186]:
df["disciplinasCursadas"] = df["disciplinasCursadas"].astype(str)
# df['disciplinasCursadas'] = df['disciplinasCursadas'].apply(lambda x: [tuple(item.split(' - ')) for item in x.split('\n')])
df["disciplinasCursadas"] = processar_disciplinas(df["disciplinasCursadas"])

In [187]:
df_exploded = df.explode("disciplinasCursadas")
df_exploded.head(5).T

,0,0,0,0,0
matriculaDRE,0,0,0,0,0
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,"(2000/1, MAA123 Algebra para Informatica, 060,...","(2000/1, MAB111 Fund da Computação Digital, 08...","(2000/1, MAB120 Computacao para Informatica, 0...","(2000/1, MAE111 Cálculo Infinitesimal I, 056, ...","(2000/1, MAE115 Cálculo Vetorial e G Analitica..."
crPorPeriodo,"[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ..."
craPorPeriodo,"[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ...","[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ...","[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ...","[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ...","[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ..."


In [188]:
df_exploded[df_exploded['matriculaDRE'] == 2742]

,matriculaDRE,periodoIngressoUFRJ,cursoIngressoUFRJ,cursoAtual,periodoIngressoCursoAtual,situacaoMatriculaAtual,sexo,disciplinasCursadas,crPorPeriodo,craPorPeriodo,periodosIntegralizados,cargaHorariaAcumulada,nome_completo,periodo_temp
2742,2742,2022/1,Ciência da Computação,Ciência da Computação,2022/1,Ativa,M,"(nan,)","[(2022/1, 0)]","[(2022/1, 0)]",1,0,Maria Julia Pires,None


In [189]:
valor_nan = df_exploded['disciplinasCursadas'].apply(lambda x: len(x) == 1)
list_dre = df_exploded[valor_nan]['matriculaDRE'].tolist()

Agora vamos pegar as informações de cada desciplina e desmembrar em novas colunas


In [190]:
def process_disciplinas(x):
    if x[0] == 'nan':
        return (np.nan, np.nan, np.nan, np.nan, np.nan)
        # return np.nan
    try:
        return (
            x[0].replace(" ", ""),
            x[1].split()[0].replace(" ", ""),
            " ".join(x[1].split()[1:]),
            x[2].replace(" ", ""),
            x[3].replace(" ", ""),
        )
    except IndexError:
        print("Erro de índice para:", x)
        return None

In [191]:
df_exploded["disciplinasCursadas"] = df_exploded["disciplinasCursadas"].apply(process_disciplinas)

In [192]:
df_disciplina = pd.DataFrame(
    df_exploded["disciplinasCursadas"].tolist(),
    index=df_exploded.index,
    columns=(
        "periodo",
        "cod_disciplina",
        "disciplina",
        "grau_disciplina",
        "situacao_disciplina",
    ),
)
df_disciplina[["matriculaDRE"]] = df_exploded[["matriculaDRE"]]
df_disciplina

,periodo,cod_disciplina,disciplina,grau_disciplina,situacao_disciplina,matriculaDRE
0,2000/1,MAA123,Algebra para Informatica,060,Aprovado,0
0,2000/1,MAB111,Fund da Computação Digital,089,Aprovado,0
0,2000/1,MAB120,Computacao para Informatica,094,Aprovado,0
0,2000/1,MAE111,Cálculo Infinitesimal I,056,Aprovado,0
0,2000/1,MAE115,Cálculo Vetorial e G Analitica,085,Aprovado,0
...,...,...,...,...,...,...
2779,2022/1,ICP111,Fund Computação Digital,067,Aprovado,2779
2779,2022/1,ICP112,Sistemas de Informação,082,Aprovado,2779
2779,2022/1,ICP120,Computação I (CC),075,Aprovado,2779
2779,2022/1,ICP136,Introd Pensamento Dedutivo,081,Aprovado,2779


In [193]:
# df_disciplina[df_disciplina['periodo'].isna()]

## Meger Colunas Tratadas


In [194]:
df_merge = pd.merge(
    df_craPeriodo, df_crPeriodo, on=["periodo", "matriculaDRE"], how="left"
)
df_merge = pd.merge(df_merge, df_disciplina, on=["periodo", "matriculaDRE"], how="outer")
df_merge

,periodo,cr_acumulado,matriculaDRE,cr_periodo,cod_disciplina,disciplina,grau_disciplina,situacao_disciplina
0,2000/1,7.6,0,7.6,MAA123,Algebra para Informatica,060,Aprovado
1,2000/1,7.6,0,7.6,MAB111,Fund da Computação Digital,089,Aprovado
2,2000/1,7.6,0,7.6,MAB120,Computacao para Informatica,094,Aprovado
3,2000/1,7.6,0,7.6,MAE111,Cálculo Infinitesimal I,056,Aprovado
4,2000/1,7.6,0,7.6,MAE115,Cálculo Vetorial e G Analitica,085,Aprovado
...,...,...,...,...,...,...,...,...
99272,2022/0,NaN,2757,NaN,MAE111,Cálculo Infinitesimal I,T,Aprovado
99273,2022/0,NaN,2757,NaN,MAE125,Álgebra Linear II,093,Aprovado
99274,2022/0,NaN,2757,NaN,MAE992,Cálculo Integ e Diferencial II,T,Aprovado
99275,2022/0,NaN,2757,NaN,MAE993,Cálculo Int e Diferencial III,T,Aprovado


In [195]:
df_merge.dropna(subset=['periodo'], inplace=True)

In [196]:
df.drop(['periodo_temp'], inplace=True, axis=1)
df_completo = pd.merge(df_merge, df, on=["matriculaDRE"], how="left")
df_completo.sample(5).T

,62927,67726,60811,68253,61021
periodo,2015/1,2014/2,2013/1,2015/2,2011/2
cr_acumulado,9,6.3,5.3,5.4,5.2
matriculaDRE,1569,1697,1518,1708,1523
cr_periodo,9.4,4.2,5.7,5.5,5.2
cod_disciplina,MAB489,MAB230,MAB355,MAB489,MAB111
disciplina,Banco de Dados I,Cálculo Numérico (CC),Arquitetura de Computadores I,Banco de Dados I,Fund da Computação Digital
grau_disciplina,089,045,060,071,054
situacao_disciplina,Aprovado,Reprovadomedia,Aprovado,Aprovado,Aprovado
periodoIngressoUFRJ,2012/1,2013/1,2011/2,2013/1,2011/2
cursoIngressoUFRJ,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação


In [197]:
df_completo[["ano", "semestre"]] = df_completo["periodo"].str.split("/", expand=True)
df_completo.head(5).T

,0,1,2,3,4
periodo,2000/1,2000/1,2000/1,2000/1,2000/1
cr_acumulado,7.6,7.6,7.6,7.6,7.6
matriculaDRE,0,0,0,0,0
cr_periodo,7.6,7.6,7.6,7.6,7.6
cod_disciplina,MAA123,MAB111,MAB120,MAE111,MAE115
disciplina,Algebra para Informatica,Fund da Computação Digital,Computacao para Informatica,Cálculo Infinitesimal I,Cálculo Vetorial e G Analitica
grau_disciplina,060,089,094,056,085
situacao_disciplina,Aprovado,Aprovado,Aprovado,Aprovado,Aprovado
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática


In [198]:
df_completo.drop(
    ["disciplinasCursadas", "crPorPeriodo", "craPorPeriodo"], axis=1, inplace=True
)

### Padronizando Nomeclatura das Colunas


In [199]:
colunas = {
    "cod_disciplina": "cod_disciplina",
    "disciplina": "nome_disciplina",
    "matriculaDRE": "matricula_dre",
    "periodoIngressoUFRJ": "periodo_ingresso_ufrj",
    "cursoIngressoUFRJ": "curso_ingresso_ufrj",
    "cursoAtual": "curso_atual",
    "situacaoMatriculaAtual": "situacao_matricula",
    "periodoIngressoCursoAtual": "periodo_ingresso_curso_atual",
    "periodosIntegralizados": "periodos_integralizados",
    "cargaHorariaAcumulada": "carga_horaria_acumulada",
}
df_completo.rename(columns=colunas, inplace=True)

In [200]:
# teste = pd.read_excel('data/raw/arquivo_anonimizado.xlsx')
# teste = teste[teste['periodoIngressoUFRJ'] != '2022/2']
# teste[teste['situacaoMatriculaAtual'] == 'Ativa']

In [201]:
# sorted(teste['situacaoMatriculaAtual'].unique())

In [202]:
# Falta o Exclusão Lógica

mapeamento_situacao = {
    'Ativa': 'Ativa',
    'Rematrícula por destrancamento automático' : 'Ativa',
    'Rematrícula por ativação do segmento referente via AGF' : 'Ativa',
    'Rematrícula por destrancamento ou descancelamento' : 'Ativa',
    'Cancelada por abandono': 'Cancelada',
    'Cancelada por abandono Definitivo': 'Cancelada',
    'Cancelada por morte': 'Cancelada',
    'Cancelada por outros motivos': 'Cancelada',
    'Cancelada por rendimento escolar insuficiente': 'Cancelada',
    'Cancelada por transferência': 'Cancelada',
    'Cancelada por ultrapassar prazo de integralização': 'Cancelada',
    'Cancelamento a pedido': 'Cancelada',
    'Cancelamento por conclusão de Mobilidade Acadêmica': 'Cancelada',
    'Cancelamento por decisão judicial': 'Cancelada',
    'Cancelamento por opção de curso': 'Cancelada',
    'Cancelamento por opção de instituição': 'Cancelada',
    'Exclusão Lógica': 'Cancelada',
    'Cancelamento por ultrapassagem do prazo máximo de trancamento': 'Cancelada',
    'Trancada': 'Trancada',
    'Trancamento Solicitado': 'Trancada',
    'Trancamento automático: Perigo de cancelamento imediato': 'Trancada',
    'Cancelada por conclusão de curso': 'Concluido'
}

df_completo['situacao_matricula'] = df_completo['situacao_matricula'].map(mapeamento_situacao)

In [203]:
df_completo.to_csv('data/trusted/arquivo_anonimizado.csv', index=False)

In [204]:
# df_completo[df_completo['periodo'] == '2022/2'].T

In [205]:
# teste = pd.read_excel('data/raw/arquivo_anonimizado.xlsx')
# print(teste[teste['matriculaDRE'] == 2510]['crPorPeriodo'].tolist())
# df_disciplina[df_disciplina['matriculaDRE'] == 2510]


# Realizando Cagas das Tabelas


## Dimenssão Aluno


In [206]:
colunas_aluno = ["sk_d_aluno", "nome_completo", "matricula_dre", "sexo"]

In [207]:
d_aluno = df_completo[colunas_aluno[1:]]

# d_aluno.groupby(['nome_completo', 'matricula_dre', 'sexo']).first().reset_index()
d_aluno = d_aluno.drop_duplicates().reset_index(drop=True)
d_aluno

,nome_completo,matricula_dre,sexo
0,Nicolas Castro,0,M
1,Pedro Rodrigues,1,M
2,Juliana Barbosa,2,M
3,Alícia Caldeira,3,M
4,Letícia Santos,4,M
...,...,...,...
2775,Ana Laura Freitas,2307,M
2776,Antônio Moraes,2371,M
2777,Pietra Moraes,2381,M
2778,Eloah Freitas,2389,M


In [208]:
d_aluno["sk_d_aluno"] = d_aluno.apply(
    lambda x: hashlib.sha256(str(x["matricula_dre"]).encode()).hexdigest(), axis=1
)
d_aluno = d_aluno.reindex(columns=colunas_aluno)
d_aluno.to_csv("data/refined/d_aluno.csv", index=False)
d_aluno

,sk_d_aluno,nome_completo,matricula_dre,sexo
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,Nicolas Castro,0,M
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,Pedro Rodrigues,1,M
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,Juliana Barbosa,2,M
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,Alícia Caldeira,3,M
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,Letícia Santos,4,M
...,...,...,...,...
2775,07aa2015d482734372d4a9a1c07c8290198526b9ce1fd2...,Ana Laura Freitas,2307,M
2776,8833ec27c2d477f05ce8764b47ae89a38c0356f70f3b9b...,Antônio Moraes,2371,M
2777,607f6bfaff663e8ec9dd623e3a16f7fe09bc170d010602...,Pietra Moraes,2381,M
2778,c81b352b270b2e498256b5e24c726a039cd665477a83d7...,Eloah Freitas,2389,M


## Dimenssão Disciplina


In [209]:
# def criar_dimenssao(nome_dimenssao, colunas_dimenssao, hash):
#     dimenssao = df_completo[colunas_dimenssao[1:]]
#     dimenssao = dimenssao.drop_duplicates().reset_index(drop=True)
#     dimenssao[f'sk_d_{nome_dimenssao}'] = d_disciplina.apply(lambda x: hashlib.sha256(str(x[hash]).encode()).hexdigest(), axis=1)
#     d_disciplina = d_disciplina.reindex(columns=colunas_aluno)
#     d_disciplina.to_csv(f'data/d_{nome_dimenssao}.csv', index=False)
#     d_disciplina

In [210]:
colunas_disciplina = ["sk_d_disciplina", "cod_disciplina", "nome_disciplina"]

In [211]:
d_disciplina = df_completo[colunas_disciplina[1:]]
d_disciplina = d_disciplina.drop_duplicates().reset_index(drop=True)
d_disciplina

,cod_disciplina,nome_disciplina
0,MAA123,Algebra para Informatica
1,MAB111,Fund da Computação Digital
2,MAB120,Computacao para Informatica
3,MAE111,Cálculo Infinitesimal I
4,MAE115,Cálculo Vetorial e G Analitica
...,...,...
1590,IEE113,Teoria Economica I
1591,MAE512,Fund Computação Científica II
1592,MAE112,Calc Vetorial e Geom Analitica
1593,MAE245,Álgebra Linear II - EQ


In [212]:
d_disciplina["sk_d_disciplina"] = d_disciplina.apply(
    lambda x: hashlib.sha256(
        str(x["cod_disciplina"] + x["nome_disciplina"]).encode()
    ).hexdigest(),
    axis=1,
)
d_disciplina = d_disciplina.reindex(columns=colunas_disciplina)
d_disciplina.to_csv("data/refined/d_disciplina.csv", index=False)
d_disciplina

,sk_d_disciplina,cod_disciplina,nome_disciplina
0,26ae25e81ccc85aecdad201a410a09485d415acd4daf6e...,MAA123,Algebra para Informatica
1,4ba51a95ba6087306319254f45d5ee5ad335922069282f...,MAB111,Fund da Computação Digital
2,09e5060ee94ea2b9604e23295927e68d40d481a362d81e...,MAB120,Computacao para Informatica
3,63637e47a78400c87bb1b82da2fc5b6c00e78c0c77c779...,MAE111,Cálculo Infinitesimal I
4,6f904cf605935af92e93713c9eead620c9c92cd971dfc4...,MAE115,Cálculo Vetorial e G Analitica
...,...,...,...
1590,4c85fe2ffb527fe500f1e7e8062b8c8eee748b69cdf8f8...,IEE113,Teoria Economica I
1591,151f406dba7091a8c887f2432d672f10ca19ca51ff1933...,MAE512,Fund Computação Científica II
1592,cd47b7d74e30795e2d4ee836bf8c0f0ee5bcd9f7de7ff7...,MAE112,Calc Vetorial e Geom Analitica
1593,574c8aa6dc1f57bf5ca26827a0871cbd70ef3416288591...,MAE245,Álgebra Linear II - EQ


## Dimenssão Periodo


In [213]:
colunas_periodo = ["sk_d_periodo", "ano", "semestre", "periodo"]

In [214]:
df_completo[df_completo['periodo'].isna()]

,periodo,cr_acumulado,matricula_dre,cr_periodo,cod_disciplina,nome_disciplina,grau_disciplina,situacao_disciplina,periodo_ingresso_ufrj,curso_ingresso_ufrj,curso_atual,periodo_ingresso_curso_atual,situacao_matricula,sexo,periodos_integralizados,carga_horaria_acumulada,nome_completo,ano,semestre


In [215]:
d_periodo = df_completo[colunas_periodo[1:]]
d_periodo = d_periodo.drop_duplicates().reset_index(drop=True)
d_periodo

,ano,semestre,periodo
0,2000,1,2000/1
1,2000,2,2000/2
2,2001,1,2001/1
3,2001,2,2001/2
4,2002,1,2002/1
...,...,...,...
68,2019,0,2019/0
69,2021,0,2021/0
70,2013,3,2013/3
71,2014,3,2014/3


In [216]:
d_periodo["sk_d_periodo"] = d_periodo.apply(
    lambda x: hashlib.sha256(str(x["ano"] + x["semestre"]).encode()).hexdigest(), axis=1
)
d_periodo = d_periodo.reindex(columns=colunas_periodo)
d_periodo.to_csv("data/refined/d_periodo.csv", index=False)
d_periodo

,sk_d_periodo,ano,semestre,periodo
0,fc82267b45dcbf8db9e4ec10055373cfff41efcbc5b83a...,2000,1,2000/1
1,29433eae6f7f1308d9799275f3a90a0afe1fef0e1818a7...,2000,2,2000/2
2,e4b8f2da99fd9407c758cd306f846562424ec5c38d3202...,2001,1,2001/1
3,b6167a59dbfdca3d716a326a015dd8a00f417590eb34e2...,2001,2,2001/2
4,c291fe829d40dc63fc4fa578f6208f425064945d31ca82...,2002,1,2002/1
...,...,...,...,...
68,8ad00ecd5eb3a6ca42f90cc776b009968ec5865f2d37b3...,2019,0,2019/0
69,fa122eae077795ac4ff0f80bb75550a9a2c30a192d035c...,2021,0,2021/0
70,fa103adf3e582291297b8330060eace6bc1a14f8737cf2...,2013,3,2013/3
71,10071c57cf3b2135572815c0243a079fc18936a1259c3d...,2014,3,2014/3


## Dimenssão Curso


In [217]:
colunas_curso = ["sk_d_curso", "curso_ingresso_ufrj", "curso_atual"]

In [218]:
d_curso = df_completo[colunas_curso[1:]]
d_curso = d_curso.drop_duplicates().reset_index(drop=True)
d_curso

,curso_ingresso_ufrj,curso_atual
0,Bacharelado em Informática,Bacharelado em Ciência da Computação
1,Bacharelado em Informática,Bacharelado em Informática
2,Bacharelado em Informática,Licenciatura em Matemática
3,Bacharelado em Informática,Mat Aplicada - Computação Científica
4,Matemática,Bacharelado em Ciência da Computação
...,...,...
74,Licenciatura em Física,Bacharelado em Ciência da Computação
75,Bacharelado em Ciência da Computação,Licenciatura em Geografia
76,Ciências Matemáticas e da Terra,Ciência da Computação
77,Bacharelado em Ciência da Computação,Engenharia Eletrônica e de Computação


In [219]:
d_curso["sk_d_curso"] = d_curso.apply(
    lambda x: hashlib.sha256(
        str(x["curso_ingresso_ufrj"] + x["curso_atual"]).encode()
    ).hexdigest(),
    axis=1,
)
d_curso = d_curso.reindex(columns=colunas_curso)
d_curso.to_csv("data/refined/d_curso.csv", index=False)
d_curso

,sk_d_curso,curso_ingresso_ufrj,curso_atual
0,74ea6a4ffce0cb8d3013a9978cfd9db27b906f4006c1f2...,Bacharelado em Informática,Bacharelado em Ciência da Computação
1,dbde55e27224a8c832119eba0c54cb25a32bf6f6417dd2...,Bacharelado em Informática,Bacharelado em Informática
2,cb369a9ae66bcd4659f91ed5d88c2a7674ca54645d641a...,Bacharelado em Informática,Licenciatura em Matemática
3,0367c21aa27e715610d936288c8fb811fc6dd378b050b5...,Bacharelado em Informática,Mat Aplicada - Computação Científica
4,5f20305be16f0143bc9a1bcc213bc104dd93a64e5ad031...,Matemática,Bacharelado em Ciência da Computação
...,...,...,...
74,b04e50e1e81b07e4bba26b99505e3a21fd9160bdd6c664...,Licenciatura em Física,Bacharelado em Ciência da Computação
75,2d37b7f8c6308826f3cd805e314628baf4dede1e70bb0a...,Bacharelado em Ciência da Computação,Licenciatura em Geografia
76,acbc30f06ec39601955703a6f4fb05791a031a8f7755d1...,Ciências Matemáticas e da Terra,Ciência da Computação
77,a54d2fed0213e2b06b6fb51bf134c8f5920420505116c3...,Bacharelado em Ciência da Computação,Engenharia Eletrônica e de Computação


## Dimenssão Situação Academica


In [220]:
# colunas_situacao_academica = ['sk_d_situacao_academica', 'situacaoMatriculaAtual']

In [221]:
# d_situacao_academica = df_completo[colunas_situacao_academica[1:]]
# d_situacao_academica = d_situacao_academica.drop_duplicates().reset_index(drop=True)
# d_situacao_academica

In [222]:
# d_situacao_academica['sk_d_situacao_academica'] = d_situacao_academica.apply(lambda x: hashlib.sha256(str(x['situacaoMatriculaAtual']).encode()).hexdigest(), axis=1)
# d_situacao_academica = d_situacao_academica.reindex(columns=colunas_situacao_academica)
# d_situacao_academica.to_csv('data/refined/d_situacao_academica.csv', index=False)
# d_situacao_academica

## Fato Desempenho Academico


In [223]:
f_desempenho_academico = df_completo.drop(
    [
        "periodo_ingresso_curso_atual",
        "periodos_integralizados",
        "carga_horaria_acumulada",
        "situacao_matricula"
    ],
    axis=1,
)
f_desempenho_academico.head(1)

,periodo,cr_acumulado,matricula_dre,cr_periodo,cod_disciplina,nome_disciplina,grau_disciplina,situacao_disciplina,periodo_ingresso_ufrj,curso_ingresso_ufrj,curso_atual,sexo,nome_completo,ano,semestre
0,2000/1,7.6,0,7.6,MAA123,Algebra para Informatica,060,Aprovado,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,M,Nicolas Castro,2000,1


In [224]:
# # D_SITUACAO_ACADEMICA
# f_desempenho_academico = pd.merge(f_desempenho_academico, d_situacao_academica, on=['situacao_matricula_atual'], how='left')
# f_desempenho_academico.drop(['situacaoMatriculaAtual'], axis=1, inplace=True)
# f_desempenho_academico.sample(5).T

In [225]:
# D_PERIODO
f_desempenho_academico = pd.merge(
    f_desempenho_academico, d_periodo, on=["ano", "semestre", "periodo"], how="left"
)
f_desempenho_academico.drop(["ano", "semestre", "periodo"], axis=1, inplace=True)
f_desempenho_academico.sample(5).T

,49953,68530,47578,56999,22798
cr_acumulado,6.4,4.4,9.4,4.8,5.5
matricula_dre,1192,1715,1126,1424,545
cr_periodo,6.4,5.6,9,6.3,5.2
cod_disciplina,FIW113,MAB111,MAB241,MAB352,MAB354
nome_disciplina,Introducao à Fisica B,Fund da Computação Digital,Computacao II,Matemática Combinatória,Computadores e Sociedade
grau_disciplina,060,064,100,061,090
situacao_disciplina,Aprovado,Aprovado,Aprovado,Aprovado,Aprovado
periodo_ingresso_ufrj,2009/1,2013/1,2008/2,2011/1,2004/1
curso_ingresso_ufrj,Bach em Ciências Matemáticas e da Terra,Bach em Ciências Matemáticas e da Terra,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
curso_atual,Bacharelado em Ciência da Computação,Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação


In [226]:
# D_CURSO
f_desempenho_academico = pd.merge(
    f_desempenho_academico,
    d_curso,
    on=["curso_ingresso_ufrj", "curso_atual"],
    how="left",
)
f_desempenho_academico.drop(
    ["curso_ingresso_ufrj", "curso_atual"], axis=1, inplace=True
)
f_desempenho_academico.sample(5).T

,46427,93137,38013,7483,61515
cr_acumulado,5.1,7.9,6.7,8.8,3.9
matricula_dre,1092,2536,898,181,1538
cr_periodo,5.1,8,7.3,8.8,2.4
cod_disciplina,MAB120,MAB624,MAB533,MAB525,MAE993
nome_disciplina,Computacao para Informatica,Números Inteiros Criptografia,Fund da Engenharia de Software,Tópicos Esp em Int Artificial,Cálculo Int e Diferencial III
grau_disciplina,052,077,050,094,000
situacao_disciplina,Aprovado,Aprovado,Aprovado,Aprovado,Reprovadomedia
periodo_ingresso_ufrj,2008/2,2020/1,2006/2,2001/1,2011/2
sexo,M,M,M,M,M
nome_completo,Pedro da Mata,Emanuelly da Costa,Caroline Campos,Alexia Costela,Rodrigo Teixeira


In [227]:
# D_ALUNO
f_desempenho_academico = pd.merge(
    f_desempenho_academico,
    d_aluno,
    on=["nome_completo", "sexo", "matricula_dre"],
    how="left",
)
f_desempenho_academico.drop(
    ["nome_completo", "sexo", "matricula_dre"], axis=1, inplace=True
)
f_desempenho_academico.sample(5).T

,5670,97684,35770,84991,83470
cr_acumulado,7.4,NaN,6.6,7.5,6.6
cr_periodo,7.7,NaN,7.4,7.5,5.4
cod_disciplina,MAB232,MAE111,IEE115,MAB624,MAB489
nome_disciplina,Programação Linear I,Cálculo Infinitesimal I,Economia,Números Inteiros Criptografia,Banco de Dados I
grau_disciplina,070,T,073,074,053
situacao_disciplina,Aprovado,Aprovado,Aprovado,Aprovado,Aprovado
periodo_ingresso_ufrj,2001/1,2011/2,2006/1,2017/1,2017/1
sk_d_periodo,5335faf37616961de2ca0d7493e78bd54539094f2fbe54...,818ca42d051ebe1ba9d7be8398fd4ff982a0c9445ee268...,b39df6b510adbc87de51b969e1b9491c3091285fde82d7...,dffcc977a35e3e941a5567f71d111418e49a25109152d0...,18457d56c04bc061fcc2fe38e7eedb79d9614385457d0d...
sk_d_curso,74ea6a4ffce0cb8d3013a9978cfd9db27b906f4006c1f2...,69f98f9a931103c9534d1881425cd98f8d920686439325...,69f98f9a931103c9534d1881425cd98f8d920686439325...,a80f5dd573274667573b525e0750e7915a3bc967190075...,a80f5dd573274667573b525e0750e7915a3bc967190075...
sk_d_aluno,13671077b66a29874a2578b5240319092ef2a1043228e4...,7ab74ac64fd9a732609f768e00ca317591dbe8d6c340bc...,55e8ab098d48f8be5578e3d3708496d152a27c4c571358...,2f8375d2a98f83f0aed5fb0d64b6e805901267915321ce...,9e3b6232cb7bd60ef9c5088e1c6dac834ddc05d916e88e...


In [228]:
# D_DISCIPLINA
f_desempenho_academico = pd.merge(
    f_desempenho_academico,
    d_disciplina,
    on=["cod_disciplina", "nome_disciplina"],
    how="left",
)
f_desempenho_academico.drop(["cod_disciplina", "nome_disciplina"], axis=1, inplace=True)
f_desempenho_academico.sample(5).T

,81604,57710,79311,93572,68349
cr_acumulado,6.2,7.8,2.9,8.4,6.3
cr_periodo,5.6,6.5,0.4,8.9,0
grau_disciplina,058,051,008,094,000
situacao_disciplina,Aprovado,Aprovado,Reprovadomedia,Aprovado,Reprfalta/media
periodo_ingresso_ufrj,2016/2,2011/1,2016/1,2020/1,2013/1
sk_d_periodo,dddd8a22f99ec19d35bd2f6c9714ba305614d47f3ca371...,e319d9b68ca50f52f6dfba34b7784ba81c72646766f52c...,dddd8a22f99ec19d35bd2f6c9714ba305614d47f3ca371...,a410ca8badb0e813980534c7b60a5425432d4ff5921a13...,70d6309c8a5c913ad3ef3800beae7ccac93955bd451810...
sk_d_curso,acbc30f06ec39601955703a6f4fb05791a031a8f7755d1...,e67cd04104eed0fa49918885c6dfb5194c965d6b690ca8...,a80f5dd573274667573b525e0750e7915a3bc967190075...,a80f5dd573274667573b525e0750e7915a3bc967190075...,69f98f9a931103c9534d1881425cd98f8d920686439325...
sk_d_aluno,c812fa553afe699033551a18bcdb76046fed094adb8c77...,d55a6ae3aba2d301e18b7bb6baa9d0ac7ce3c14300361a...,158a323a7ba44870f23d96f1516dd70aa48e9a72db4ebb...,5ee36f40051aec9076cd20826e9bad65d1c5fa24dc02a6...,e41fbda961c9b0a19e1232a464bdf5a03c5b5568b12147...
sk_d_disciplina,310f5c6e86c7e6cbe976fa5706be1805c8d1f8b59bd6ca...,7d0088e71ff447c97905637ed9a41f282f353ae0038680...,2993d0e1bb6ecd5e29879451e7463a05cc6075da0e70d7...,4ba51a95ba6087306319254f45d5ee5ad335922069282f...,e2742bc95ef811e079e1c09e811746699fdc585b34d5dd...


In [229]:
f_desempenho_academico.drop(["periodo_ingresso_ufrj"], axis=1, inplace=True)

In [230]:
f_desempenho_academico.sample(5).T

,93813,1932,28206,29976,34068
cr_acumulado,8.3,6.6,8.7,6.3,7.7
cr_periodo,7.5,7.7,8.4,10,8.3
grau_disciplina,061,076,073,100,096
situacao_disciplina,Aprovado,Aprovado,Aprovado,Aprovado,Aprovado
sk_d_periodo,418f67a6d7651a50985b8e9885a16184e3e34867ebcce7...,551f1b6da3d076f00cf0f65b0bbb08ee6e88d26efd4abf...,c8b0f0d28b77989b0224197cdd95ba9c797d1d96d60ca0...,fb191b5a87e994fc286b26efeef99c68ea71d461a82821...,ed0661bd6e1c613c15349c48cc1986e69c99cac57db13a...
sk_d_curso,a80f5dd573274667573b525e0750e7915a3bc967190075...,74ea6a4ffce0cb8d3013a9978cfd9db27b906f4006c1f2...,69f98f9a931103c9534d1881425cd98f8d920686439325...,69f98f9a931103c9534d1881425cd98f8d920686439325...,69f98f9a931103c9534d1881425cd98f8d920686439325...
sk_d_aluno,8fc750f6d387270884c52388abf261efa9e0b55854fdd2...,aea92132c4cbeb263e6ac2bf6c183b5d81737f179f21ef...,172e1676eda470ede17e9d491554bcbe97ba4691f92880...,92c5fd0421c1d619cbf1bdba83a207261f2c5f764aed46...,fce86e339dc3131c489202ec3b6c8d4319c61f152b3541...
sk_d_disciplina,2341fc8810a7dc5ecabdbf386b9107ce4cfcb9ff3e879a...,b94f2e3c271228b4550d73f47d22d378e30c48557a76f7...,0d19d67fdde9ff3848a3ef8e302b307d23b8c4a560c19b...,12f769113c5c3e546a86a14a514096838da7d7ea4c392b...,6443551721c8799599f041a85bb54b1fd00b0780332460...


In [231]:
f_desempenho_academico.to_csv("data/refined/f_desempenho_academico.csv", index=False)

## Fato Matricula Aluno


In [232]:
f_matricula_aluno = df_completo.drop(
    [
        "cod_disciplina",
        "nome_disciplina",
        "grau_disciplina",
        "situacao_disciplina",
        "cr_acumulado",
        "cr_periodo",
    ],
    axis=1,
)
f_matricula_aluno.head(1)

,periodo,matricula_dre,periodo_ingresso_ufrj,curso_ingresso_ufrj,curso_atual,periodo_ingresso_curso_atual,situacao_matricula,sexo,periodos_integralizados,carga_horaria_acumulada,nome_completo,ano,semestre
0,2000/1,0,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,2000/1,Concluido,M,13,3105,Nicolas Castro,2000,1


In [233]:
f_matricula_aluno = (
    f_matricula_aluno.groupby(
        [
            "matricula_dre",
            "situacao_matricula",
            "periodos_integralizados",
            "carga_horaria_acumulada",
            "curso_ingresso_ufrj",
            "curso_atual",
        ]
    )["periodo"]
    .max()
    .reset_index()
)
f_matricula_aluno[["ano", "semestre"]] = f_matricula_aluno["periodo"].str.split("/", expand=True)

In [234]:
f_matricula_aluno.head(2)

,matricula_dre,situacao_matricula,periodos_integralizados,carga_horaria_acumulada,curso_ingresso_ufrj,curso_atual,periodo,ano,semestre
0,0,Concluido,13,3105,Bacharelado em Informática,Bacharelado em Ciência da Computação,2006/1,2006,1
1,1,Concluido,10,3330,Bacharelado em Informática,Bacharelado em Ciência da Computação,2004/1,2004,1


In [235]:
teste = pd.read_excel('data/raw/arquivo_anonimizado.xlsx')

In [236]:
f_matricula_aluno[f_matricula_aluno['matricula_dre'] == 1563]

,matricula_dre,situacao_matricula,periodos_integralizados,carga_horaria_acumulada,curso_ingresso_ufrj,curso_atual,periodo,ano,semestre
1563,1563,Ativa,2,2850,Bacharelado em Ciência da Computação,Ciência da Computação,2022/1,2022,1


In [237]:
teste[teste['matriculaDRE'] == 1563]

,Unnamed: 0,matriculaDRE,periodoIngressoUFRJ,cursoIngressoUFRJ,cursoAtual,periodoIngressoCursoAtual,situacaoMatriculaAtual,sexo,disciplinasCursadas,crPorPeriodo,craPorPeriodo,periodosIntegralizados,cargaHorariaAcumulada,CRA,periodosTrancados,periodosCancelados,periodosCRMenor3,reprovacoes
1563,1563,1563,2012/1,Bacharelado em Ciência da Computação,Ciência da Computação,2021/1,Rematrícula por destrancamento ou descancelamento,M,2012/1 - MAB112 Sistemas de Informação - 069 -...,2012/1 - 3.5\n2012/2 - 4.9\n2013/1 - 5.1\n2013...,2012/1 - 3.5\n2012/2 - 4.1\n2013/1 - 4.4\n2013...,2,2850,5.6,2021/2,NaN,2020/2 | 2022/1,2012/1 - MAB120 Computação I (CC) - 014 - Repr...


In [238]:
df_completo[df_completo['matricula_dre'] == 1563]

,periodo,cr_acumulado,matricula_dre,cr_periodo,cod_disciplina,nome_disciplina,grau_disciplina,situacao_disciplina,periodo_ingresso_ufrj,curso_ingresso_ufrj,curso_atual,periodo_ingresso_curso_atual,situacao_matricula,sexo,periodos_integralizados,carga_horaria_acumulada,nome_completo,ano,semestre
62538,2012/1,3.5,1563,3.5,MAB112,Sistemas de Informação,069,Aprovado,2012/1,Bacharelado em Ciência da Computação,Ciência da Computação,2021/1,Ativa,M,2,2850,Daniel Moreira,2012,1
62539,2012/1,3.5,1563,3.5,MAB120,Computação I (CC),014,Reprovadomedia,2012/1,Bacharelado em Ciência da Computação,Ciência da Computação,2021/1,Ativa,M,2,2850,Daniel Moreira,2012,1
62540,2012/1,3.5,1563,3.5,MAB624,Números Inteiros Criptografia,070,Aprovado,2012/1,Bacharelado em Ciência da Computação,Ciência da Computação,2021/1,Ativa,M,2,2850,Daniel Moreira,2012,1
62541,2012/1,3.5,1563,3.5,MAE111,Cálculo Infinitesimal I,000,Reprovadomedia,2012/1,Bacharelado em Ciência da Computação,Ciência da Computação,2021/1,Ativa,M,2,2850,Daniel Moreira,2012,1
62542,2012/2,4.1,1563,4.9,MAB113,Organização da Informação,062,Aprovado,2012/1,Bacharelado em Ciência da Computação,Ciência da Computação,2021/1,Ativa,M,2,2850,Daniel Moreira,2012,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62594,2020/1,5.6,1563,9.6,MAB122,Computação Gráfica I,096,Aprovado,2012/1,Bacharelado em Ciência da Computação,Ciência da Computação,2021/1,Ativa,M,2,2850,Daniel Moreira,2020,1
62595,2020/1,5.6,1563,9.6,MAB125,Projeto de Jogos,097,Aprovado,2012/1,Bacharelado em Ciência da Computação,Ciência da Computação,2021/1,Ativa,M,2,2850,Daniel Moreira,2020,1
62596,2020/2,5.5,1563,2,MAB515,Avaliação e Desempenho,020,Reprfalta/media,2012/1,Bacharelado em Ciência da Computação,Ciência da Computação,2021/1,Ativa,M,2,2850,Daniel Moreira,2020,2
62597,2021/1,5.6,1563,9.3,MAB515,Avaliação e Desempenho,093,Aprovado,2012/1,Bacharelado em Ciência da Computação,Ciência da Computação,2021/1,Ativa,M,2,2850,Daniel Moreira,2021,1


In [239]:
# D_ALUNO
f_matricula_aluno = pd.merge(
    f_matricula_aluno,
    d_aluno,
    on=["matricula_dre"],
    how="inner",
)
f_matricula_aluno.drop(
    ["nome_completo", "sexo", "matricula_dre"], axis=1, inplace=True
)
f_matricula_aluno.sample(2)

,situacao_matricula,periodos_integralizados,carga_horaria_acumulada,curso_ingresso_ufrj,curso_atual,periodo,ano,semestre,sk_d_aluno
1731,Concluido,12,3290,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,2019/2,2019,2,05758cd3875ad2171484c0026ccbb8adc210cd2d852407...
1678,Ativa,3,2940,Bacharelado em Ciência da Computação,Ciência da Computação,2022/1,2022,1,18bed1232c7d6375b760860a0c1904bf560a92b8cec096...


In [240]:
# D_CURSO
f_matricula_aluno = pd.merge(
    f_matricula_aluno,
    d_curso,
    on=["curso_ingresso_ufrj", "curso_atual"],
    how="inner",
)
f_matricula_aluno.drop(
    ["curso_ingresso_ufrj", "curso_atual"], axis=1, inplace=True
)
f_matricula_aluno.sample(2)

,situacao_matricula,periodos_integralizados,carga_horaria_acumulada,periodo,ano,semestre,sk_d_aluno,sk_d_curso
1800,Cancelada,1,0,2019/1,2019,1,3451d35d093f0572f939572b1d8fa2a20a41cf62f5b592...,69f98f9a931103c9534d1881425cd98f8d920686439325...
786,Concluido,18,3210,2015/0,2015,0,1a1cf797fabe7f95836fabeca626907c77b3e6c9aff7c2...,69f98f9a931103c9534d1881425cd98f8d920686439325...


In [241]:
f_matricula_aluno.to_csv("data/refined/f_matricula_aluno.csv", index=False)

# Analise de disciplinas com nomes errados


In [242]:
d_disciplina[d_disciplina["cod_disciplina"] == "MAB613"]

,sk_d_disciplina,cod_disciplina,nome_disciplina
51,69c74b0dc4bd99f8f39728f26b5edb757d2f12f54e08ea...,MAB613,Met Desenv de Sist Informacao
109,4831abbc49e3963e443cfe77fde3e1f4966cbaa7adf118...,MAB613,Met Desenv de Sist Informação


In [243]:
d_disciplina[d_disciplina["cod_disciplina"] == "MAB601"]
# EEL410

,sk_d_disciplina,cod_disciplina,nome_disciplina
50,fa1851c7178a5ca652bdbb609397732777b444e5be4aa4...,MAB601,Top Esp em Eng de Softaware
61,e4b4dc9868db99e7a7c2f8c49d7597179bcf9697daf319...,MAB601,Top Esp em Eng de Software


In [244]:
d_disciplina[d_disciplina["cod_disciplina"].duplicated()]

,sk_d_disciplina,cod_disciplina,nome_disciplina
61,e4b4dc9868db99e7a7c2f8c49d7597179bcf9697daf319...,MAB601,Top Esp em Eng de Software
109,4831abbc49e3963e443cfe77fde3e1f4966cbaa7adf118...,MAB613,Met Desenv de Sist Informação
341,bb19397c608b8ad6e1e91f353d8aeaa3c0109a13a33ecb...,MAB603,Gestão do Conhecimento
362,58ae4c22500c1bf1a3dd156b6405367f685508d5a856ab...,MAB230,Cálculo Numérico (CC)
395,70c5438681f6efe2902a4b9e78fd4488412a85544fdf14...,MAB602,Dw no Sup à Tomada de Decisão
397,c6fdafe7054dc9166dd654d7d98ef1d8da16718f40133a...,MAB600,Dados Semi-estruturados e XML
433,ad46eeb3845a2b7de8e5ceebc4230a1850be2509cad643...,MAB532,Mineração de Dados
435,a8cff0b3f3cdc6991275d971f5fef3fdfbbf8d6393afbf...,EEH210,Engenharia e Meio Ambiente
444,6350703008dfb26f834d42c1683bf89f50315ddbea870e...,IEE304,Desenvolvimento Socioeconomico
445,310f5c6e86c7e6cbe976fa5706be1805c8d1f8b59bd6ca...,MAB120,Computação I (CC)
